PyTorch 문법을 내가 너무 모르고 있다.. 문법에 대한 이해를 통해 소스코드를 완전히 내것으로 만들자.
date: 230113

In [1]:
import torch
from torch import nn, optim

import random
import numpy as np
import torch.nn.functional as F

In [2]:
def build_model(input_size: int):
    model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, 64),
        nn.ReLU(),
        nn.Linear(64, 64),
        nn.ReLU(),
        nn.Linear(64, 4),
    )
    return model

In [3]:
model = build_model(16) # state size는 16이다
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=16, out_features=64, bias=True)
  (2): ReLU()
  (3): Linear(in_features=64, out_features=64, bias=True)
  (4): ReLU()
  (5): Linear(in_features=64, out_features=4, bias=True)
)

In [4]:
# 히스토리 데이터가 3개 있다
states = torch.tensor(np.zeros((3, 16), dtype=np.float32))
states

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [9]:
ret = model(states) # type: torch.Tensor
print(ret.shape, type(ret))
ret # 3개의 히스토리에서, 상하좌우로 액션했을 때의 Q값

torch.Size([3, 4]) <class 'torch.Tensor'>


tensor([[-0.0562,  0.1440,  0.0749, -0.1187],
        [-0.0562,  0.1440,  0.0749, -0.1187],
        [-0.0562,  0.1440,  0.0749, -0.1187]], grad_fn=<AddmmBackward0>)

In [6]:
actions = torch.tensor([[1], [1], [2]]) # 각각 하,하,좌로 이동했음
qurrent_q = ret.gather(1, actions) # 각 액션했을 때의 Q값

In [39]:
b = np.zeros((3,4), dtype=np.float32)
b

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)

In [40]:
b.reshape((1, -1))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [52]:
target_model = build_model(16)

# 지금 막 실행한 하나의 state
next_states = torch.tensor(np.array([np.zeros((16), dtype=np.float32)]))
print(next_states.shape)
ret = target_model(next_states) # type: torch.Tensor
ret # 지금 막 실행한 상태에서 상하좌우 각각의 기대값

torch.Size([1, 16])


tensor([[-0.1410,  0.0518,  0.0096, -0.1373]], grad_fn=<AddmmBackward0>)

In [62]:
# 다음 상태에서의 최대 기대값
max_next_q = ret.detach().max(1)[0]

tensor([0.0518])